##### Copyright 2020 The TensorFlow Quantum Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Meta-Learning for QAOA

In this notebook you will explore the application of meta-learning techniques from [here](https://arxiv.org/abs/1907.05415) to improve initialization of QAOA.


Authors : Michael Broughton, Antonio J. Martinez

Contributors: Guillaume Verdon

Created : 2020-Feb-06

Last updated : 2020-Apr-09

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/tensorflow/quantum/blob/research/metalearning_qaoa/metalearning_qaoa.ipynb)

## Import dependencies

In [0]:
!pip install --upgrade tensorflow==2.3.1

In [0]:
!pip install tensorflow-quantum

In [0]:
import cirq
import networkx as nx
import numpy as np
import random
import sympy
import tensorflow as tf
import tensorflow_quantum as tfq
np.random.seed(123)
random.seed(123)

## QAOA



The QAOA ansatz consists of repeated applications of a mixer Hamiltonian $\hat{H}_M$ and the cost Hamiltonian $\hat{H}_C$.  The total applied unitary is
$$\hat{U}(\eta,\gamma) = \prod_{j=1}^{p}e^{-i\eta_{j}\hat{H}_M}e^{-i\gamma_{j} \hat{H}_C},$$
where $p$ is the number of times the mixer and cost are applied; the parameters $\eta_j, \gamma_j$ are to be optimized to produce a bitstring of minimal energy with respect to $\hat{H}_C$.

One traditional family of Hamiltonians used in QAOA are the Ising models.  These are defined as
$$\hat{H}_\mathrm{P}=\sum_i h_i \hat{Z}^{(i)}+\sum_{i,j} J_{ij} \hat{Z}^{(i)}\hat{Z}^{(j)}.$$
There is a one-to-one mapping between weighted graphs and Ising models: $h_i$ can be thought of as the weight of a graph node $i$ and $J_{ij}$ can be thought of as the weight of a graph edge between nodes $i$ and $j$.  In applications such as [MaxCut](https://en.wikipedia.org/wiki/Maximum_cut), we have $h_i = 0$ and $J_{ij} = 1$ for all indices $i$ and $j$.  The importance of this graph correspondence motivates us to define the following function, which takes a graph and returns the corresponding Ising model:

In [0]:
def maxcut_qaoa_from_graph(graph, p):
  qubits = cirq.GridQubit.rect(1, len(graph.nodes))
  qaoa_circuit = cirq.Circuit()
  # Initial equal superposition
  for qubit in qubits:
    qaoa_circuit += cirq.H(qubit)
  qaoa_symbols = []
  # Stack the parameterized costs and mixers
  for l_num in range(p):
    qaoa_symbols.append(sympy.Symbol("gamma_{}".format(l_num)))
    for e in graph.edges():
      qaoa_circuit += cirq.ZZ(qubits[e[0]], qubits[e[1]])**qaoa_symbols[-1]
    qaoa_symbols.append(sympy.Symbol("eta_{}".format(l_num)))
    for n in graph.nodes():
      qaoa_circuit += cirq.X(qubits[n])**qaoa_symbols[-1]
  # Define the cost as a Cirq PauliSum
  cost_op = None
  for e in graph.edges():
    if cost_op is None:
      cost_op = cirq.Z(qubits[e[0]])*cirq.Z(qubits[e[1]])
    else:
      cost_op += cirq.Z(qubits[e[0]])*cirq.Z(qubits[e[1]])
  return qaoa_circuit, qaoa_symbols, cost_op

## Meta-Learning for MaxCut

The idea of meta-learning for optimization is to train an outer-loop optimizer on many instances of a problem class, to enhance the efficiency of solving unseen instances.  In other words, the learner is attempting to extract the common structure among instances of a particular class of problems.

Here, you will use a recurrent neural network to find good initial parameter settings for MaxCut QAOA instances.  As shown in the [original paper](https://arxiv.org/abs/1907.05415), this is an effective method for overcoming the challenge of ["barren plateaus"](https://www.nature.com/articles/s41467-018-07090-4) in quantum machine learning.

To this end we define a function that generates a set of random MaxCut QAOA instances, based on graphs sampled from an [Erdős–Rényi](https://en.wikipedia.org/wiki/Erd%C5%91s%E2%80%93R%C3%A9nyi_model) distribution:

In [0]:
def generate_data(n_nodes, n_points):
  datapoints = []
  costs = []
  for _ in range(n_points):
    random_graph = nx.gnp_random_graph(n_nodes, p=3. / n_nodes)
    circuit, symbols, cost_op = maxcut_qaoa_from_graph(random_graph, 1)
    datapoints.append(circuit)
    costs.append([cost_op])
  return datapoints, symbols, costs

Since our recurrent neural network will have both classical and quantum components, we will need to define a custom RNN layer:

In [0]:
class QRNN(tf.keras.layers.Layer):
  def __init__(self, symbol_names):
    super(QRNN, self).__init__()
    self.shared = tf.keras.layers.Dense(25, name="shared")
    self.state = tf.keras.layers.Dense(25, name="state")
    self.params = tf.keras.layers.Dense(2, name="params")
    self.expectation = tfq.layers.Expectation()
    self.symbol_names = symbol_names

  def call(self, inputs):
    circuits = inputs[0]
    ops = inputs[1]
    state = inputs[2]
    params = inputs[3]
    prev_output = inputs[4]
    joined = tf.keras.layers.concatenate([state, params, prev_output])
    shared = self.shared(joined)
    s_inp = self.state(shared)
    p_inp = self.params(shared)
    exp_out = self.expectation(circuits,
                               symbol_names=self.symbol_names,
                               symbol_values=p_inp,
                               operators=ops)
    return [circuits, ops, s_inp, p_inp, exp_out]

This layer is stacked to produce the meta-learner RNN.  We choose 5 shots of optimization:

In [0]:
# Generate random MaxCut instances as training data.
N_QUBITS = 10

# For a more accurate optimizer on testing data, increase N_POINTS
N_POINTS = 500
circuits, symbols, ops = generate_data(N_QUBITS, N_POINTS)
circuit_tensor = tfq.convert_to_tensor(circuits)
ops_tensor = tfq.convert_to_tensor(ops)

# Unroll the RNN through time.
state_inp = tf.keras.Input(shape=(25,))
params_inp = tf.keras.Input(shape=(2,))
exp_inp = tf.keras.Input(shape=(25,))

op_inp = tf.keras.Input(shape=(1,), dtype=tf.dtypes.string)
circuit_inp = tf.keras.Input(shape=(), dtype=tf.dtypes.string)

rnn_0 = QRNN(symbols)
rnn_1 = QRNN(symbols)
rnn_2 = QRNN(symbols)
rnn_3 = QRNN(symbols)
rnn_4 = QRNN(symbols)
output_0 = rnn_0([circuit_inp, op_inp, state_inp, params_inp, exp_inp])
output_1 = rnn_1(output_0)
output_2 = rnn_2(output_1)
output_3 = rnn_3(output_2)
output_4 = rnn_4(output_3)

Now we can set up a loss function over the 5 timesteps of our RNN QAOA optimizer:

In [0]:
@tf.function
def value_loss(unused, outputs):
    return tf.reduce_mean(outputs)

# It's important to have a good guess on the last shot of the optimization
loss = tf.keras.layers.average([
    0.1 * output_0[4], 0.2 * output_1[4], 0.3 * output_2[4],
    0.4 * output_3[4], 0.5 * output_4[4]
])

# Penalize jumping around randomly in the landscape.
penalizer = 10 * tf.reduce_sum(
    (output_0[3] - output_1[3])**2 + (output_1[3] - output_2[3])**2 +
    (output_2[3] - output_3[3])**2 + (output_3[3] - output_4[3])**2,
    axis=1)
full_loss = loss + penalizer

Finally we set and train our full Keras model:

In [0]:
# Can change these to random along with longer tine horizon and greater training
# data for more robust test set performance
initial_state = np.zeros((N_POINTS, 25)).astype(np.float32)
initial_params = np.zeros((N_POINTS, 2)).astype(np.float32)
initial_exp = np.zeros((N_POINTS, 25)).astype(np.float32)

# Our model will output it's parameter guesses along with the loss value that is
# computed over them. This way we can use the model to guess parameters later on
model = tf.keras.Model(inputs=[circuit_inp, op_inp, state_inp, params_inp, exp_inp],
    outputs=[
        output_0[3], output_1[3], output_2[3], output_3[3], output_4[3],
        full_loss
    ])
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss=value_loss, loss_weights=[0, 0, 0, 0, 0, 1])

model.fit(x=[circuit_tensor, ops_tensor, initial_state, initial_params, initial_exp],
          y=[
              np.zeros((N_POINTS, 1)),
              np.zeros((N_POINTS, 1)),
              np.zeros((N_POINTS, 1)),
              np.zeros((N_POINTS, 1)),
              np.zeros((N_POINTS, 1)),
              np.zeros((N_POINTS, 1))
          ],
          epochs=20,
          batch_size=64,
          verbose=1)

(Doing validation data.)

In [0]:
circuits, parameters, ops = generate_data(10, N_POINTS // 2)

circuit_tensor = tfq.convert_to_tensor(circuits)
ops_tensor = tfq.convert_to_tensor(ops)

initial_state = np.zeros((N_POINTS // 2, 25)).astype(np.float32)
initial_guesses = np.zeros((N_POINTS // 2, 2)).astype(np.float32)
initial_exp = np.zeros((N_POINTS // 2, 25)).astype(np.float32)

out1, out2, out3, out4, out5, _ = model(
    [circuit_tensor, ops_tensor, initial_state, initial_guesses, initial_exp])

one_vals = tf.reduce_mean(tfq.layers.Expectation()(
    circuit_tensor,
    symbol_names=parameters,
    symbol_values=out1,
    operators=ops_tensor)).numpy()
two_vals = tf.reduce_mean(tfq.layers.Expectation()(
    circuit_tensor,
    symbol_names=parameters,
    symbol_values=out2,
    operators=ops_tensor)).numpy()
three_vals = tf.reduce_mean(tfq.layers.Expectation()(
    circuit_tensor,
    symbol_names=parameters,
    symbol_values=out3,
    operators=ops_tensor)).numpy()
four_vals = tf.reduce_mean(tfq.layers.Expectation()(
    circuit_tensor,
    symbol_names=parameters,
    symbol_values=out4,
    operators=ops_tensor)).numpy()
five_vals = tf.reduce_mean(tfq.layers.Expectation()(
    circuit_tensor,
    symbol_names=parameters,
    symbol_values=out5,
    operators=ops_tensor)).numpy()

average_cost_function_values = [
    one_vals, two_vals, three_vals, four_vals, five_vals
]
std_of_param_guess = [
    tf.math.reduce_std(out1).numpy(),
    tf.math.reduce_std(out2).numpy(),
    tf.math.reduce_std(out3).numpy(),
    tf.math.reduce_std(out4).numpy(),
    tf.math.reduce_std(out5).numpy()
]

print('-' * 80)
print('Average cost function values for each guess number across unseen instances:',
      average_cost_function_values)
print('Variance of parameter values guessed:', std_of_param_guess)

Explore a singular instances. Now this instance is on 12 qubits and not just 10.

In [0]:
test_graph_circuit, parameters, test_graph_op = generate_data(12, 1)

test_graph_circuit = test_graph_circuit[0]
test_graph_op = test_graph_op[0][0]

resolution = 100
input_vals = []
for i, a in enumerate(np.linspace(-0.5, .5, resolution)):
    for j, b in enumerate(np.linspace(-0.5, .5, resolution)):
        input_vals.append([a, b])

cost_vals = tfq.layers.Expectation()(test_graph_circuit,
                                     symbol_names=parameters,
                                     symbol_values=np.array(input_vals),
                                     operators=test_graph_op).numpy()

output_vals = np.empty((resolution, resolution))
for i, a in enumerate(np.linspace(-0.5, 0.5, resolution)):
    for j, b in enumerate(np.linspace(-0.5, 0.5, resolution)):
        output_vals[i][j] = cost_vals[i * resolution + j]

import matplotlib.pyplot as plt

plt.imshow(output_vals)

guess_0, guess_1, guess_2, guess_3, guess_4, _ = model([
    tfq.convert_to_tensor([test_graph_circuit]),
    tfq.convert_to_tensor([[test_graph_op]]),
    np.zeros((1, 25)).astype(np.float32),
    np.zeros((1, 2)).astype(np.float32),
    np.zeros((1, 25)).astype(np.float32),
])
all_guesses = [guess_0, guess_1, guess_2, guess_3, guess_4]
all_guesses = [list(a.numpy()[0]) for a in all_guesses]


# This should be cleaned up...
def f(x):
    sim = cirq.Simulator()
    final_state = sim.simulate(test_graph_circuit, {
        parameters[0]: x[0],
        parameters[1]: x[1]
    }).final_state
    q = sorted(list(test_graph_circuit.all_qubits()))
    res = test_graph_op.expectation_from_wavefunction(
        final_state, qubit_map={h: i for i, h in enumerate(q)}).real
    return res


all_costs = [f(a) for a in all_guesses]

plt.plot((np.array(all_guesses)[:, 0] + 0.5) * resolution,
         (np.array(all_guesses)[:, 1] + 0.5) * resolution,
         c='r',
         linestyle='--',
         markevery=[4],
         marker='*',
         markersize=20,
         linewidth=3.5)
plt.show()

print('All guesses for test graph:', all_guesses)
print('Cost function values for test graph:', all_costs)
print('-' * 80)

As we can see from the above visual the RNN immediately begins guessing near the basin of attraction and continues to explore around the region looking to improve the estimate. Note the values in `all_costs` decreasing.